### Retrieval Augmented Generation Using MongoDb backend

### Install the required packages

In [ ]:
#%pip install python-dotenv
#%pip install requests
#%pip install azure-storage-blob
#%pip install tabulate

### Load the configuration settings

In [ ]:
from dotenv import dotenv_values, load_dotenv
import sys
sys.path.append('../common')
from helper import Helper

load_dotenv()

secrets = dotenv_values("secrets.env")
api_key = secrets.get("API_KEY")
local_env = secrets.get('LOCAL_URL') or secrets.get("APIM_NAME")
environment = secrets.get("APIM_NAME")
account_connection_string = secrets.get("AZURE_STORAGE_CONNECTION_STRING")
region_override = secrets.get("REGION_OVERRIDE")
X_KPMG_END_USER_JWT = secrets.get("X_KPMG_END_USER_JWT")

ragHelper = Helper(local_env, "rag/kpmg", api_key, region_override)
deploymentsHelper = Helper(environment, "genai/azure/deployments", api_key, region_override)

if X_KPMG_END_USER_JWT:
    ragHelper.add_headers({"x-kpmg-end-user-jwt": X_KPMG_END_USER_JWT})
    


### Health Check Tests

In [ ]:
response = ragHelper.get("/health-check/self")

assert response.status_code == 200
assert response.json['status'] == "Healthy"

### Version Tests

In [ ]:
response = ragHelper.get("/version")

assert response.status_code == 200
assert response.json['version'].startswith("1.0")

In [ ]:
# clean up any previous runs
response = ragHelper.get("/catalog")

assert response.status_code == 200

if (len(response.json["value"])):
    for catalog in response.json["value"]:
        response = ragHelper.delete(f"/catalog/{catalog['catalogId']}")
        assert response.status_code == 200

### Catalog Tests

##### Create a new catalog

In [ ]:
import datetime

catalogName = f'Regression Catalog: {datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S")}'

ragHelper.log("Catalog Name: " + catalogName + '\n')

response = ragHelper.post("/catalog", {
    "displayName": catalogName,
    "backendDatabase": "mongodb"
})

assert response.status_code == 200

catalogId = response.json["catalogId"]

ragHelper.log("Created Catalog Id: " + catalogId)

##### Generate a SAS URI for the catalog source

In [ ]:
from azure.storage.blob import BlobServiceClient, generate_container_sas, ContainerSasPermissions
from datetime import datetime, timedelta, timezone

source_container_name = 'reg-source'
directory_name = 'sourceData'

blob_service_client = BlobServiceClient.from_connection_string(account_connection_string)
account_name = blob_service_client.account_name

source_sas_token = generate_container_sas(
    account_name=account_name,
    container_name=source_container_name,
    account_key=blob_service_client.credential.account_key,
    permission=ContainerSasPermissions(read=True, list=True),
    expiry=datetime.now(timezone.utc) + timedelta(hours=24)
)

source_sas_uri = f"https://{account_name}.blob.core.windows.net/{source_container_name}/{directory_name}?{source_sas_token}"

ragHelper.log("Source SAS URI: " + source_sas_uri)

##### Create Catalog Source

In [ ]:
catalogSourceName = f'Regression Catalog Source'

response = ragHelper.post(f"/catalog/{catalogId}/source/blob", {
    "displayName": catalogSourceName,
    "uri": source_sas_uri
})

assert response.status_code == 200

sourceId = response.json["sourceId"]

ragHelper.log("Created Catalog Source Id: " + sourceId)

##### Start an Ingestion Job

In [ ]:
response = ragHelper.post(f"/catalog/{catalogId}/ingest", None)

assert response.status_code == 202
assert 'ingestionJobId' in response.json

ingestionJobId = response.json["ingestionJobId"]

ragHelper.log("Created Ingestion Job Id: " + ingestionJobId)

##### Get the Status of the Job

In [ ]:
import time

ingestionStatus = 'Ingesting'
waitTime = 5

while (ingestionStatus != 'Ingested' and ingestionStatus != 'IngestionFailed'):
    time.sleep(waitTime)
    response = ragHelper.get(f"/catalog/{catalogId}/ingest/{ingestionJobId}")

    ingestionStatus = response.json["ingestionStatus"]

    if (waitTime < 30):
        # backoff polling wait time
        # the longer we wait, the less frequently we poll
        waitTime = waitTime + 5

ragHelper.log(f"Ingestion Job Status: {ingestionStatus}")

assert response.status_code == 200
assert ingestionStatus == 'Ingested'

##### Get Ingestion Job File Statuses

In [ ]:
from tabulate import tabulate

response = ragHelper.get(f"/catalog/{catalogId}/ingest/{ingestionJobId}/files")

file_list = ragHelper.get_property_values(response.json, 'fileName')

print(tabulate(response.json['value'], headers="keys", tablefmt="grid"))

##### Get ingestion jobs for a catalog

In [ ]:
response = ragHelper.get(f"/catalog/{catalogId}/ingest")

assert response.status_code == 200

##### Get Catalog Source Files

In [ ]:
from tabulate import tabulate

response = ragHelper.get(f"/catalog/{catalogId}/source/{sourceId}/files")

file_list = ragHelper.get_property_values(response.json, 'fileName')

print(tabulate(response.json['value'], headers="keys", tablefmt="grid"))

##### Search the index

In [ ]:
from IPython.display import display, Markdown

response = ragHelper.post(f"/catalog/{catalogId}/search", {
    'count': 1,
    'search': 'Who is the lead for Azure Data Factory best practices?'
})

assert len(response.json['value']) == 1

chunkFileContent = response.json['value'][0]['chunkFileContent']

display(Markdown(chunkFileContent))

#### Get a catalog Source

In [ ]:
response = ragHelper.get(f"/catalog/{catalogId}/source/{sourceId}")

assert response.status_code == 200

##### Delete the Catalog Source

In [ ]:
response = ragHelper.delete(f"/catalog/{catalogId}/source/{sourceId}")

assert response.status_code == 200

##### List Catalog Sources

In [ ]:
response = ragHelper.get(f"/catalog/{catalogId}/source")

assert response.status_code == 404
assert response.json["message"] == "The catalog sources are either not found or you are not the creator."

#### Get a Catalog

In [ ]:
response = ragHelper.get(f"/catalog/{catalogId}")

assert response.status_code == 200

##### Delete the Catalog

In [ ]:
response = ragHelper.delete(f"/catalog/{catalogId}")

assert response.status_code == 200

##### List Catalogs

In [ ]:
response = ragHelper.get("/catalog")

assert response.status_code == 200
assert len(response.json["value"]) == 0